# STEP 0: Install dependencies

In [ ]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [ ]:
import os
import getpass
from langgraph.graph import StateGraph,END # Import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [ ]:
os.environ['GOOGLE_API_KEY']=getpass.getpass('Enter your Gemini API key: ')

Enter your Gemini API key: ··········


# STEP 2: Initialize Gemini 1.5 Flash

In [ ]:
llm=ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest",temperature=0.2)

# STEP 3: Node to ask user for symptom

In [ ]:
def get_symptom(state: dict) ->dict:
  symptom=input("Welcome to XYZ Hospital,Please enter your symptom")
  state["symptom"]=symptom
  return state

# STEP 4: Node to classify the symptom

In [ ]:
def classify_symptom(state: dict) ->dict:
  prompt=("You are a helpful Medical Assistant,classify the symptoms below into one of the categories \n"
           "-General \n -Emergency \n -mental health \n"
           f"symptom: {state['symptom']}\n"
           "Respond only with one word :General,Emergency or Mental Heath"
           "#Example :input : I have fever ,Output : General"
  )
  response=llm.invoke(prompt)
  category=response.content.strip()
  print(f"LLM classifies the symptom as : {category}")
  state["category"]=category
  return state

# STEP 5: Router logic to route to the correct node


In [ ]:
def symptom_router(state: dict) ->dict:
  cat=state["category"].lower()
  if "general" in cat:
    return "general"
  elif "Emergency" in cat:
    return "Emergency"
  elif "mental health" in cat:
    return "Mental Health"
  else:
    return "general"



# STEP 6: Category-specific response nodes


In [ ]:
def general_node(state: dict) ->dict:
  state["answer"] = f"  '{state['symptom']}' : seems general:directing you to general ward for consulting a doctor"
  return state

def emergency_node(state: dict) ->dict:
  state["answer"] = f"  '{state['symptom']}' : it is a medical emergency:seeking immediate medical help"
  return state

def mental_health_node(state: dict) ->dict:
  state["answer"] = f"  '{state['symptom']}' : seems like a medical health issue:talk to our counsellor"
  return state

# STEP 7: Build LangGraph


In [ ]:
# Create the builder
builder = StateGraph(dict)

# Define all nodes
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify_symptom", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("Emergency", emergency_node)
builder.add_node("Mental Health", mental_health_node)

# Add transitions (edges)
builder.add_edge("get_symptom", "classify_symptom")

builder.add_conditional_edges("classify_symptom", symptom_router, {
    "general": "general",
    "Emergency": "Emergency",
    "Mental Health": "Mental Health"
})

# Add END nodes
builder.add_edge("general", END)
builder.add_edge("Emergency", END)
builder.add_edge("Mental Health", END)

# Set the entry point
builder.set_entry_point("get_symptom")

# STEP 8: Compile and invoke the graph


In [ ]:
graph = builder.compile()
final_state=graph.invoke({})
print("final output \n")
print(final_state["answer"])


Welcome to XYZ Hospital,Please enter your symptomi have dust allergy
LLM classifies the symptom as : General
final output 

  'i have dust allergy' : seems general:directing you to general ward for consulting a doctor
